In [1]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [20]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input:
    artist_ids.append(input[0][link][32:54])

'4tZwfgrHOc3mvqYlEYSvVi'

In [2]:
artist_uri = 'spotify:artist:6eJa3zG1QZLRB3xgRuyxbm'
# https://open.spotify.com/playlist/1G6DJ7kNFGop94rmlpMoYW?si=842f9dda4b934bcc
results = sp.artist_albums(artist_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

album_ids = []
for album in albums:
    album_ids.append(album['id'])

album_track_ids = []
for album in album_ids:

    results = sp.album_tracks(album)
    tracks = results['items']
    track_ids = []
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)   

album_track_ids

[['3CzfTl1RSsQ6jnxKZmsJMe',
  '0CWvHfEwbeBVN0RViTybvL',
  '3W0fmsdPgPisEQEfefimGT',
  '4oB6hNxskthsaZB1OuxgZX',
  '0brBOx8ejPXvBYnQkWquYK',
  '76rMrk4t6Cmg2EXR7VV8xs',
  '3R12JORSoLUDpSzpS8semL',
  '1c8g7AOUIdREi458yJNIdq',
  '0PGnwDPPMoJ1s49hQQXHFR',
  '0iN2lUhuFEjQ5hHGXDO7ET',
  '5VwI6NilYghaY9PfP91GlG'],
 ['3IxKc2OLHw5ltmUkQKoUkV',
  '64lsIF5pw0sJY0gV5kz0RN',
  '0zePbRMJ9sd7wsZRlbPQua',
  '0AsUVUloJ52ZCXmqrMFgOI',
  '09u08pFFRhzoVuleMAgtFq',
  '6pjBrkKAwZn8qC77Lqags7',
  '57HJa4ktE7iHGrbFhwzEr2',
  '6H3vlGfkDO5M2WTDMNbdoI',
  '2KfFmPKssYjVxmzUmin1rI',
  '5LEVjmo2QweDHc7k19aFcp']]

In [3]:
album_track_ids[0]

['3CzfTl1RSsQ6jnxKZmsJMe',
 '0CWvHfEwbeBVN0RViTybvL',
 '3W0fmsdPgPisEQEfefimGT',
 '4oB6hNxskthsaZB1OuxgZX',
 '0brBOx8ejPXvBYnQkWquYK',
 '76rMrk4t6Cmg2EXR7VV8xs',
 '3R12JORSoLUDpSzpS8semL',
 '1c8g7AOUIdREi458yJNIdq',
 '0PGnwDPPMoJ1s49hQQXHFR',
 '0iN2lUhuFEjQ5hHGXDO7ET',
 '5VwI6NilYghaY9PfP91GlG']

In [10]:
import xarray as xr
columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']


# Create DataFrames of features from track ids
i = 0
# final_data = xr.DataArray()
# for album in album_track_ids:


features = []
for track in album_track_ids[0]:
    results = sp.audio_features(track)
    features.append(results[0])
data1 = pd.DataFrame.from_records(features)

headings = list(data1.columns)


data1.drop(columns = columns_to_remove, axis = 1, inplace = True)
data1 = data1.to_xarray()
features = []
for track in album_track_ids[1]:
    results = sp.audio_features(track)
    features.append(results[0])
data2 = pd.DataFrame.from_records(features)


# album_name = albums[i]['name'] # need to fix special characters

# Optionally save each album to csv
# data.to_csv('./data/' + album_name + '.csv', encoding='utf-8')
# i += 1
data2.drop(columns = columns_to_remove, axis = 1, inplace = True)

# data2 = data2.to_xarray()
data2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.680,0.584,11,-10.459,1,0.0617,0.0304,0.000945,0.1060,0.770,149.976,3IxKc2OLHw5ltmUkQKoUkV,277200,4
1,0.641,0.842,9,-7.270,1,0.0292,0.1320,0.910000,0.4190,0.500,129.980,64lsIF5pw0sJY0gV5kz0RN,278769,4
2,0.707,0.767,7,-7.072,1,0.0323,0.0279,0.000295,0.1690,0.832,139.985,0zePbRMJ9sd7wsZRlbPQua,204014,4
3,0.706,0.631,0,-7.711,1,0.0553,0.1580,0.000607,0.0374,0.424,160.064,0AsUVUloJ52ZCXmqrMFgOI,176250,4
4,0.772,0.700,9,-7.699,1,0.0454,0.0505,0.051500,0.1470,0.942,127.990,09u08pFFRhzoVuleMAgtFq,262500,4
5,0.436,0.381,0,-10.920,1,0.0425,0.9480,0.017600,0.1130,0.331,88.160,6pjBrkKAwZn8qC77Lqags7,142125,4
6,0.488,0.510,7,-8.367,1,0.0260,0.0233,0.150000,0.1430,0.368,104.959,57HJa4ktE7iHGrbFhwzEr2,219429,4
7,0.263,0.521,8,-8.022,1,0.0465,0.0835,0.047400,0.0891,0.263,74.983,6H3vlGfkDO5M2WTDMNbdoI,252400,4
8,0.752,0.531,0,-12.875,1,0.0369,0.0681,0.004800,0.2930,0.892,110.018,2KfFmPKssYjVxmzUmin1rI,203727,4
9,0.694,0.583,11,-11.549,1,0.0371,0.1680,0.819000,0.3290,0.897,134.999,5LEVjmo2QweDHc7k19aFcp,215556,4


In [9]:
# Save data to csv in subfolder
# final_data = xr.DataArray(data)

# name_da = xr.DataArray([da.name for da in (data1, data2)], name='varname')
final_data = xr.concat([data1, data2], dim = ["data1", "data2"])
final_data
# final_data.to_netcdf("saved_on_disk.nc")

<xarray.Dataset>
Dimensions:           (index: 11, concat_dim: 2)
Coordinates:
  * index             (index) int64 0 1 2 3 4 5 6 7 8 9 10
  * concat_dim        (concat_dim) <U5 'data1' 'data2'
Data variables: (12/14)
    danceability      (concat_dim, index) float64 0.817 0.866 ... 0.694 nan
    energy            (concat_dim, index) float64 0.661 0.754 ... 0.583 nan
    key               (concat_dim, index) float64 0.0 0.0 11.0 ... 0.0 11.0 nan
    loudness          (concat_dim, index) float64 -4.835 -5.531 ... -11.55 nan
    mode              (concat_dim, index) float64 0.0 1.0 0.0 ... 1.0 1.0 nan
    speechiness       (concat_dim, index) float64 0.102 0.0356 ... 0.0371 nan
    ...                ...
    liveness          (concat_dim, index) float64 0.162 0.0801 ... 0.329 nan
    valence           (concat_dim, index) float64 0.945 0.962 ... 0.897 nan
    tempo             (concat_dim, index) float64 140.0 115.0 ... 135.0 nan
    id                (concat_dim, index) object '3CzfTl1RSsQ6jnxKZmsJMe' ......
    duration_ms       (concat_dim, index) float64 1.143e+05 2.292e+05 ... nan
    time_signature    (concat_dim, index) float64 4.0 4.0 4.0 ... 4.0 4.0 nan

In [16]:
# Live
sp.audio_features('2RpXUwCytDFqZSHl6kIquB')

[{'danceability': 0.26,
  'energy': 0.99,
  'key': 7,
  'loudness': -3.498,
  'mode': 1,
  'speechiness': 0.128,
  'acousticness': 0.00202,
  'instrumentalness': 0.00416,
  'liveness': 0.779,
  'valence': 0.0885,
  'tempo': 128.29,
  'type': 'audio_features',
  'id': '2RpXUwCytDFqZSHl6kIquB',
  'uri': 'spotify:track:2RpXUwCytDFqZSHl6kIquB',
  'track_href': 'https://api.spotify.com/v1/tracks/2RpXUwCytDFqZSHl6kIquB',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RpXUwCytDFqZSHl6kIquB',
  'duration_ms': 349733,
  'time_signature': 4}]

In [103]:
# Remastered
sp.audio_features('0xzhryP1AoHUazYdJ5rj3B')

[{'danceability': 0.46,
  'energy': 0.922,
  'key': 2,
  'loudness': -7.388,
  'mode': 1,
  'speechiness': 0.264,
  'acousticness': 0.0449,
  'instrumentalness': 0.000252,
  'liveness': 0.34,
  'valence': 0.328,
  'tempo': 119.961,
  'type': 'audio_features',
  'id': '0xzhryP1AoHUazYdJ5rj3B',
  'uri': 'spotify:track:0xzhryP1AoHUazYdJ5rj3B',
  'track_href': 'https://api.spotify.com/v1/tracks/0xzhryP1AoHUazYdJ5rj3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xzhryP1AoHUazYdJ5rj3B',
  'duration_ms': 311027,
  'time_signature': 4}]

In [104]:
# Non-remastered (original)
sp.audio_features('4BB3Itr3YETOKFURnC28Ld')

[{'danceability': 0.472,
  'energy': 0.859,
  'key': 2,
  'loudness': -11.866,
  'mode': 1,
  'speechiness': 0.181,
  'acousticness': 0.0793,
  'instrumentalness': 0.000202,
  'liveness': 0.645,
  'valence': 0.322,
  'tempo': 119.989,
  'type': 'audio_features',
  'id': '4BB3Itr3YETOKFURnC28Ld',
  'uri': 'spotify:track:4BB3Itr3YETOKFURnC28Ld',
  'track_href': 'https://api.spotify.com/v1/tracks/4BB3Itr3YETOKFURnC28Ld',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4BB3Itr3YETOKFURnC28Ld',
  'duration_ms': 310667,
  'time_signature': 4}]